<a href="https://colab.research.google.com/github/parus-cristatus/tolokapizza/blob/main/tolokakit/ownload_attachments_with_input_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install toloka-kit

In [14]:
import toloka.client as toloka
import os
import json
import shutil

In [ ]:
toloka_client = toloka.TolokaClient(input('Enter your token:'), 'SANDBOX') #PRODUCTION
print(toloka_client.get_requester())

In [20]:
output_field_name = 'audio_file'
pool_id='1155555'

assignment_list = toloka_client.get_assignments(pool_id=pool_id)

names = set()

# create directory
os.makedirs('attachments', exist_ok=True)

In [21]:
def make_dir(assignment_id: str):
    os.makedirs(f'attachments/{assignment_id}', exist_ok=True)

def download_attachment(name: str, ext: str, att_id: str, dir_: str = ''):
    with open(f'./attachments/{dir_}{name}.{ext}', 'wb') as out_f:
        toloka_client.download_attachment(attachment_id=att_id, out=out_f)

def change_name(names: set, file_name: str, name_as_id: bool = False) -> str:
    if name_as_id:
        if file_name in names:
            file_name = attachment.id
    else:
        if file_name in names:
            file_name += f"_{attachment.id[-5:]}"

    return file_name

def remove_files(rmdirs: bool = False):
    dir_names = next(os.walk(f'./attachments/'))[1]
    if rmdirs:
        for name in dir_names:
            if name != '.ipynb_checkpoints':
                files = next(os.walk(f'./attachments/{name}/'))[2]
                for file_ in files:
                    os.remove(f'./attachments/{name}/{file_}')
            os.rmdir(path=f'./attachments/{name}')

    files = next(os.walk(f'./attachments/'))[2]
    for file_ in files:
        os.remove(f'./attachments/{file_}')

In [22]:
for assignment in assignment_list:
    input_values = {}
    make_dir(assignment.id)
    dir_path = f'{assignment.id}/'

    for item in zip(assignment.solutions, assignment.tasks):
        attachment_id = item[0].output_values.get(output_field_name)
        input_value = item[1].input_values
        input_values[attachment_id] = input_value

        attachment = toloka_client.get_attachment(attachment_id=attachment_id)
        name, extension = attachment.name.split('.')
        names.add(name)

        new_name = change_name(names=names, file_name=name, name_as_id=False)

        download_attachment(name=new_name, ext=extension, att_id=attachment_id, dir_=dir_path)
    
    with open(file=f'./attachments/{dir_path}inputs.json', mode='w', encoding='utf-8') as f:
        json.dump(input_values, f, indent=4)

In [ ]:
# make zip
shutil.make_archive(base_name='attachments', format='zip', root_dir='attachments')

In [23]:
# remove all files from /attachments directory
remove_files(rmdirs=True)

In [17]:
# Download all attachments in "attachments" directory
results_list = [attachment for attachment in toloka_client.get_attachments(pool_id=pool_id)]

for attachment in results_list:
    name, extension = attachment.name.split('.')

    names.add(name)
    new_name = change_name(names=names, file_name=name, name_as_id=True)
    # make_dir(attachment.details.assignment_id)
    # dir_path = f'{attachment.details.assignment_id}/'

    download_attachment(name=new_name, ext=extension, att_id=attachment.id)